## Importing and Installing Required Libraries

In [24]:
%pip install -qU deepagents
%pip install -qU langgraph
%pip install -qU langchain-openai
%pip install -qU pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [157]:
import os
import base64
import requests
import pandas as pd
import json
from pathlib import Path

from deepagents import create_deep_agent
from langchain_openai import ChatOpenAI


### Establishing a Connection with GridDB

In [158]:
username = os.environ.get("username")
password = os.environ.get("password")
base_url = os.environ.get("base_url")


url = f"{base_url}/checkConnection"

credentials = f"{username}:{password}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()

headers = {
    'Content-Type': 'application/json',  # Added this header to specify JSON content
    'Authorization': f'Basic {encoded_credentials}',
    'User-Agent': 'PostmanRuntime/7.29.0'
}

response = requests.get(url, headers=headers)

print(response.status_code)
print(response.text)


200



## Creating a LangGraph Deep Agent

### Simple Example of DeepAgent Working

In [ ]:
def search_db(query: str) -> str:
    """
    This tool searches the GridDB database using the provided SQL query.
    """

    print("search_db query=", query)

system_instructions = """

<role>
You are an SQL expert and you have access to a GridDB database. You can use the tool below to search the database using SQL queries. 
When you receive a user query, you should formulate an appropriate SQL query to retrieve the relevant information from the database.
</role>

<tools>
** search_db**: 
- This tool allows you to search the GridDB database using SQL queries.
- Input to this tool should be a valid SQL query string.
- The output of this tool will be the results of the SQL query.
</tools>
"""

llm = ChatOpenAI(model="gpt-4o", 
                 temperature=0)

agent = create_deep_agent(
    tools=[search_db],
    instructions=system_instructions,
    model=llm
)

# Invoke the agent
result = agent.invoke({"messages": [{"role": "user", 
                                     "content": "Select the patients who spent more than 5 days in the hospital."}]})

search_db query= SELECT * FROM patients WHERE days_in_hospital > 5;


### Defining tools for GridDB SQL Agent

#### Tools for Inserting CSV file into GridDB

In [160]:
def insert_csv_to_griddb(csv_file_path: str) -> str:

    """
    This tool inserts data from a CSV file into the GridDB database.

    Args:
        csv_file_path (str): The path to the CSV file to be inserted.
    """

    try:

        if not os.path.isfile(csv_file_path):
            return f"Error: The file '{csv_file_path}' does not exist."

        dataset = pd.read_csv(csv_file_path)

        container_name = Path(csv_file_path).stem

        ## =============================
        ## Creating Container for GridDB
        ## =============================

        dataset.insert(0, "SerialNo", dataset.index + 1)
        dataset.columns.name = None   
        # Mapping pandas dtypes to GridDB types
        type_mapping = {
            "int64":          "LONG",
            "float64":        "DOUBLE",
            "bool":           "BOOL",
            'datetime64': "TIMESTAMP", 
            "object":         "STRING",
            "category":       "STRING",
        }

        # Generate the columns part of the payload dynamically
        columns = []
        for col, dtype in dataset.dtypes.items():
            griddb_type = type_mapping.get(str(dtype), "STRING")  # Default to STRING if unknown
            columns.append({
                "name": col,
                "type": griddb_type
            })

        url = f"{base_url}/containers"
        # Create the payload for the POST request
        payload = json.dumps({
            "container_name": container_name,
            "container_type": "COLLECTION",
            "rowkey": True,  # Assuming the first column as rowkey
            "columns": columns
        })

        # Make the POST request to create the container
        response = requests.post(url, headers=headers, data=payload)

        print("Create container response:", response.status_code, response.text)
        if response.status_code != 201:
            return f"Error creating container: {response.text}"
        
        ## =============================
        ## Inserting data in the container
        ## =============================


        url = f"{base_url}/containers/{container_name}/rows"

        def format_row(row):
            formatted = []
            for item in row:
                if pd.isna(item):
                    formatted.append(None)  # Convert NaN to None
                elif isinstance(item, bool):
                    formatted.append(str(item).lower())  # Convert True/False to true/false
                elif isinstance(item, (int, float)):
                    formatted.append(item)  # Keep integers and floats as they are
                else:
                    formatted.append(str(item))  # Convert other types to string
            return formatted

        # Prepare rows with correct formatting
        rows = [format_row(row) for row in dataset.values.tolist()]

        # Create payload as a JSON string
        payload = json.dumps(rows)

        # Make the PUT request to add the rows to the container
        response = requests.put(url, headers=headers, data=payload)

        if response.status_code != 200:
            return f"Error inserting data: {response.text}"                 

        
        return f"Data inserted successfully in the container {container_name}"
        

    except Exception as e:
        return f"Error: {str(e)}"

#### Tool for Retrieving Container Columns from GridDB

In [209]:
def get_container_columns(container_name: str) -> list[str] | str:

    """
    Fetches one row from the container and tries to get column names from response metadata
    Use this tool before executing any CRUD query to get the column names
    """
    try:
        if not container_name:
            return "Error: container_name must be provided"
        
        url = f"{base_url}/containers/{container_name}/rows"
        
        payload = {
            "offset": 0,
            "limit": 1,
            "condition": "",
            "sort": ""
        }
        response = requests.post(url, headers=headers, json=payload)
        if response.status_code != 200:
            return f"Error fetching rows for container {container_name}: {response.status_code} {response.text}"
        
        data = response.json()
        
        rows = data.get("rows", None)
        if rows is None or len(rows) == 0:
            return f"No rows returned from container {container_name}"
        
        # Try to get “columns” metadata from response, if present
        if "columns" in data and isinstance(data["columns"], list):
            cols_meta = data["columns"]
            # cols_meta: list of dicts with at least "name"
            names = [col_meta.get("name", "") for col_meta in cols_meta]
            return names
        
    except Exception as e:
        return f"Error: {str(e)}"
    


#### Tool for Retrieving Data from GridDB

In [162]:

def sql_select_from_griddb(container_name: str, sql_stmt: str) -> str:
    """
    Execute a SQL SELECT query on a GridDB Cloud container via Web API,
    and return a formatted string of the results.

    If the result is an aggregate (e.g. COUNT, SUM etc.) returning a single value,
    returns something like "Aggregate result: 42".

    Otherwise, returns record by record detail.
    """
    print("sql_select_from_griddb query =", sql_stmt)
    
    try:
        if not container_name:
            return "Error: container_name must be provided"
        if not sql_stmt.strip():
            return "Error: SQL statement must be provided"
        
        url = f"{base_url}/sql"
        payload = [
            {
                "type": "sql-select",
                "stmt": sql_stmt
            }
        ]
        response = requests.post(url, headers=headers, json=payload)
        if response.status_code != 200:
            return f"Error executing SQL: {response.status_code}, {response.text}"
        
        resp_json = response.json()
        if not isinstance(resp_json, list) or len(resp_json) < 1:
            return f"Unexpected response format: {resp_json}"
        
        first = resp_json[0]
        columns_meta = first.get("columns")
        results = first.get("results")
        
        # Handle case of aggregate queries (when there are columns but rows are empty OR rows missing)
        # Or when results is present but format is such that we have just one value
        if columns_meta is not None and results is not None:
            # If only one column and one row, could be an aggregate
            if len(results) == 1 and len(columns_meta) == 1:
                # e.g. [{"name":"count", "type":"LONG"}] and [[42]]
                col_name = columns_meta[0].get("name", "value")
                val = results[0][0]
                return f"{col_name}: {val}"
            
            # Otherwise, return record by record
            output_lines = []
            for idx, row in enumerate(results, start=1):
                output_lines.append(f"=============================")
                output_lines.append(f"Record {idx}")
                for col_meta, cell in zip(columns_meta, row):
                    col_name = col_meta.get("name", "UnknownColumn")
                    cell_str = "None" if cell is None else str(cell)
                    output_lines.append(f"{col_name}: {cell_str}")
                output_lines.append("")  # blank line between records
            output = "\n".join(output_lines)
            return output
        
        # If no "results", but maybe "columns" and maybe some other field like "rows" or "rows/records"
        # Fallback: inspect other fields
        # For example, if the response has a field "rows" (old format) or "values"
        # you can try to fetch those.
        
        # If still nothing meaningful:
        return f"No usable result data found for query: {first}"

    except Exception as e:
        return f"Error: {e}"


#### Tool for Inserting and Updating Data in GridDB

In [163]:
def sql_insert_update_griddb(container_name: str, sql_stmt: str) -> str:
    """
    Execute an SQL INSERT or UPDATE query on GridDB Cloud via Web API,
    return a message indicating success or error.
    
    Args:
        container_name (str): Name of the container/table (used for readability/logging).
        sql_stmt (str): The full SQL INSERT or UPDATE statement to execute.
    
    Returns:
        A string: either success message or error.
    """
    print("sql_update_griddb query =", sql_stmt)
    
    try:
        if not container_name:
            return "Error: container_name must be provided"
        if not sql_stmt.strip():
            return "Error: SQL statement must be provided"
        
        # Construct the URL for SQL update (inserts or updates)
        url = f"{base_url}/sql/update"
        
        payload = [
            {
                "stmt": sql_stmt
            }
        ]
        
        response = requests.post(url, headers=headers, json=payload)
        
        if response.status_code != 200:
            return f"Error executing SQL update: {response.status_code}, {response.text}"
        
        # The response may or may not include useful JSON; check and return appropriate message
        try:
            resp_json = response.json()
        except ValueError:
            # Not JSON; maybe empty or plaintext
            return f"SQL update executed successfully for container '{container_name}'."
        
        # If JSON and maybe return shows how many rows affected or similar
        # Depending on what the API returns; adapt as needed
        if isinstance(resp_json, list) and len(resp_json) > 0:
            # Some APIs give back something like [{"count": N}] or status info
            # Try to find a count or status field
            first = resp_json[0]
            if "count" in first:
                return f"Successfully updated/inserted {first['count']} rows into '{container_name}'."
            else:
                # If no 'count', just return the JSON for debugging
                return f"SQL update successful. Response: {resp_json}"
        else:
            return f"SQL update successful into container '{container_name}'."
    
    except Exception as e:
        return f"Error: {e}"


#### Tool for Deleting Data From GridDB

In [ ]:
def sql_delete_rows_griddb(container_name: str, row_keys: list, convert_to_int: bool = True) -> str:
    """
    Delete one or more rows from a GridDB container via Web API.

    Args:
        container_name (str): the container to delete from.
        row_keys (list): list of rowkey values (as str or int). This can be retrieved by first calling the selectsql_select_from_griddb tool to get the rowkeys.
        convert_to_int (bool): if True, try converting keys to ints, else keep as given.

    Returns:
        A message string: success or detailed error.
    """
    try:
        if not container_name:
            return "Error: container_name must be provided"
        if not row_keys or not isinstance(row_keys, list):
            return "Error: row_keys must be a non-empty list"
        
        # Convert types if needed
        if convert_to_int:
            # only convert those that are numeric strings
            new_keys = []
            for k in row_keys:
                try:
                    ki = int(k)
                    new_keys.append(ki)
                except Exception:
                    # If conversion fails, keep original
                    new_keys.append(k)
            row_keys_to_use = new_keys
        else:
            row_keys_to_use = row_keys

        print("sql_delete_rows_griddb using row_keys =", row_keys_to_use)

        url = f"{base_url}/containers/{container_name}/rows"
        response = requests.delete(url, headers=headers, json=row_keys_to_use)

        print("DELETE response status:", response.status_code)
        print("DELETE response text:", response.text)

        # Acceptable success statuses
        if response.status_code not in (200, 204):
            return f"Error deleting rows: {response.status_code}, {response.text}"

        return f"Successfully requested delete of {len(row_keys_to_use)} row(s) from '{container_name}'."
    except Exception as e:
        return f"Error: {e}"


### Create a Deep Agent for Tool Calling

In [ ]:

system_instructions = """

<role>
You are an SQL expert and you have access to a GridDB database. You can use the tool below to search the database using SQL queries. 
When you receive a user query, you should formulate an appropriate SQL query to retrieve the relevant information from the database.
</role>

<tools>
** insert_csv_to_griddb**: 
- This tool allows you to insert data from a CSV file into the GridDB database.
- Input to this tool should be the path to a valid CSV file or the name of the CSV file. The tool will add the .csv extension if not provided.
- The tool returns a success message or an error message.

** get_container_columns**:
- This tool fetches the name of all columns in the specified container.
- Input to this tool should be the name of the container (table).
- The tool returns a list of column names or an error message.

** sql_select_from_griddb**:
- This tool allows you to search the GridDB database using SQL queries.
- Input to this tool should be a valid SQL query string.
- The output of this tool will be the results of the SQL query.

** sql_insert_update_griddb**:
- This tool allows you to insert or update data in the GridDB database using SQL queries.
- Input to this tool should be a valid SQL INSERT or UPDATE query string.
- The output of this tool will be a success message or an error message.

** sql_delete_rows_griddb**:
- This tool allows you to delete one or more rows from a GridDB container using their rowkey values.
- Input to this tool should be the name of the container (table) and a list of rowkey values identifying the rows to delete.
- Always retrieve the rowkeys by first calling the `sql_select_from_griddb` tool to get the rowkeys.
- The output of this tool will be a success message or an error message.

</tools>


<constraints>
- When using the insert_csv_to_griddb tool, ensure that the CSV file exists in the current working directory or provide the full path to the file.
- Always use the get_container_columns tool to retrieve column names before constructing SQL queries to avoid errors.
- Ensure that SQL queries are syntactically correct and reference existing containers and columns in the database.
- Before using the sql_delete_rows_griddb tool, always retrieve the rowkeys by first calling the sql_select_from_griddb tool to get the rowkeys.
</constraints>

"""

llm = ChatOpenAI(model="gpt-4o", 
                 temperature=0)

agent = create_deep_agent(
    tools=[insert_csv_to_griddb, 
           get_container_columns,
           sql_select_from_griddb,
           sql_insert_update_griddb,
           sql_delete_rows_griddb],
    instructions=system_instructions,
    model=llm
)



### Testing the GridDB SQL Agent

In [166]:
def get_response(query: str) -> str:
    # Invoke the agent
    result = agent.invoke({"messages": [{"role": "user", 
                                         "content": query}]})
    return result['messages'][-1].content

In [185]:
response = get_response("Insert the titanic_dataset.csv file into the database")
print(response)

Create container response: 201 
The `titanic_dataset.csv` file has been successfully inserted into the database. If you need any further assistance or queries regarding this dataset, feel free to ask!


In [186]:
response = get_response("What are the column names in the titanic_dataset table?")
print(response)

The column names in the `titanic_dataset` table are:

1. SerialNo
2. PassengerId
3. Survived
4. Pclass
5. Name
6. Sex
7. Age
8. SibSp
9. Parch
10. Ticket
11. Fare
12. Cabin
13. Embarked


In [187]:
response = get_response("Get the names and ages of the top 3 oldest passengers in the titanic_dataset")
print(response)


sql_select_from_griddb query = SELECT Name, Age FROM titanic_dataset WHERE Age IS NOT NULL ORDER BY Age DESC LIMIT 3
The names and ages of the top 3 oldest passengers in the Titanic dataset are:

1. **Barkworth, Mr. Algernon Henry Wilson** - Age: 80.0
2. **Svensson, Mr. Johan** - Age: 74.0
3. **Artagaveytia, Mr. Ramon** - Age: 71.0


In [188]:
response = get_response("What is the percentage of passengers who survived in the titanic_dataset?")
print(response)


sql_select_from_griddb query = SELECT (COUNT(Survived) FILTER (WHERE Survived = 1) * 100.0 / COUNT(Survived)) AS Survival_Percentage FROM titanic_dataset
sql_select_from_griddb query = SELECT (SUM(CASE WHEN Survived = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(Survived)) AS Survival_Percentage FROM titanic_dataset
The percentage of passengers who survived in the Titanic dataset is approximately 38.38%.


In [189]:

df = pd.read_csv("titanic_dataset.csv")

# Drop any rows where "Survived" is missing (if applicable)
df2 = df.dropna(subset=["Survived"])

total = len(df2)
survived = df2["Survived"].sum()  # assuming Survived is 1 for survived, 0 for not

percentage_survived = survived / total * 100

print(f"Total passengers: {total}")
print(f"Number who survived: {survived}")
print(f"Percentage who survived: {percentage_survived:.2f}%")

Total passengers: 891
Number who survived: 342
Percentage who survived: 38.38%


In [190]:
response = get_response("""
                        Insert a new passenger in the titanic_dataset. 
                        He is a 89 years old male named "John Doe",
                        with passenger class 3, embarked from Southampton,
                        having 0 siblings/spouses aboard, 0 parents/children aboard,
                        ticket number "A123", fare 7.25, cabin as NULL
                        """
)
print(response)


sql_update_griddb query = INSERT INTO titanic_dataset (PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked) VALUES (NULL, NULL, 3, 'John Doe', 'male', 89, 0, 0, 'A123', 7.25, NULL, 'S')
sql_select_from_griddb query = SELECT MAX(SerialNo) AS MaxSerialNo FROM titanic_dataset
sql_update_griddb query = INSERT INTO titanic_dataset (SerialNo, PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked) VALUES (892, NULL, NULL, 3, 'John Doe', 'male', 89, 0, 0, 'A123', 7.25, NULL, 'S')
The new passenger "John Doe" has been successfully added to the `titanic_dataset` with a `SerialNo` of 892. If you need any further assistance, feel free to ask!


In [191]:
response = get_response("Who is the oldest passenger in the titanic_dataset?")
print(response)


sql_select_from_griddb query = SELECT Name, Age FROM titanic_dataset WHERE Age IS NOT NULL ORDER BY Age DESC LIMIT 1
The oldest passenger in the Titanic dataset is John Doe, who was 89 years old.


In [192]:
response = get_response("Update the age of the oldest passenger in the titanic_dataset to 95")
print(response)

sql_select_from_griddb query = SELECT MAX(Age) AS OldestAge FROM titanic_dataset
sql_update_griddb query = UPDATE titanic_dataset SET Age = 95 WHERE Age = 89
The age of the oldest passenger in the titanic_dataset has been successfully updated to 95. If you have any more requests or need further assistance, feel free to ask!


In [193]:
response = get_response("Who is the oldest passenger in the titanic_dataset?")
print(response)

sql_select_from_griddb query = SELECT * FROM titanic_dataset WHERE Age = (SELECT MAX(Age) FROM titanic_dataset)
The oldest passenger in the Titanic dataset is John Doe, a 95-year-old male. He was in the third class, with a ticket number A123, and embarked from Southampton (Embarked: S).


In [206]:
response = get_response("Delete all the passengers younger than 30 years in the titanic_dataset")
print(response)

sql_select_from_griddb query = SELECT SerialNo FROM titanic_dataset WHERE Age < 30
sql_delete_rows_griddb using row_keys = [1, 3, 8, 9, 10, 11, 13, 15, 17, 23, 24, 25, 28, 35, 38, 39, 40, 42, 44, 45, 50, 51, 52, 54, 57, 58, 59, 60, 61, 64, 67, 68, 69, 70, 72, 73, 74, 76, 79, 81, 82, 84, 85, 87, 89, 90, 91, 92, 94, 98, 101, 103, 106, 107, 112, 113, 114, 115, 116, 118, 119, 120, 121, 126, 128, 132, 134, 135, 136, 137, 139, 140, 142, 143, 144, 145, 146, 147, 148, 152, 157, 163, 164, 165, 166, 170, 172, 173, 174, 176, 183, 184, 185, 192, 193, 194, 200, 201, 205, 206, 208, 209, 211, 213, 217, 221, 222, 226, 227, 228, 229, 232, 234, 235, 238, 239, 243, 244, 247, 248, 252, 256, 262, 267, 268, 272, 279, 282, 283, 284, 288, 290, 291, 292, 294, 295, 297, 298, 303, 306, 308, 311, 312, 313, 314, 316, 317, 321, 322, 324, 330, 334, 337, 341, 342, 343, 344, 346, 349, 351, 353, 354, 356, 357, 362, 370, 371, 372, 373, 374, 375, 377, 378, 379, 380, 382, 386, 387, 390, 392, 393, 394, 395, 396, 399, 400, 

In [207]:
response = get_response("what is the total number of passengers in the titanic_dataset?")
print(response)

sql_select_from_griddb query = SELECT COUNT(PassengerId) FROM titanic_dataset
The total number of passengers in the titanic_dataset is 507.
